In [1]:
import pyrealsense2 as rs
import numpy as np

import pygame
from pygame.locals import *

from OpenGL.GL import *
from OpenGL.GLU import *

import math

In [2]:
def draw_pointcloud(points, width, height):
    
    #glPopMatrix()
    #glPushAttrib(GL_ALL_ATTRIB_BITS)

    glClearColor(153 / 255, 153 / 255, 153 / 255, 1)
    glClear(GL_DEPTH_BUFFER_BIT)

    #glMatrixMode(GL_PROJECTION)
    #glPushMatrix()
    gluPerspective(60, width / height, 0.01, 10.0)
    
    #glMatrixMode(GL_MODELVIEW)
    #glPushMatrix()
    gluLookAt(0, 0, 0, 0, 0, 1, 0, -1, 0)

    #glTranslatef(0, 0, 0.5 + app_state.offset_y * 0.05)
    #glRotated(app_state.pitch, 1, 0, 0)
    #glRotated(app_state.yaw, 0, 1, 0)
    glTranslatef(0, 0, -0.5)

    glPointSize(width / 640)
    glEnable(GL_DEPTH_TEST)
    glEnable(GL_TEXTURE_2D)
    #glBindTexture(GL_TEXTURE_2D, app_state.tex.get_gl_handle())
    tex_border_color = [0.8, 0.8, 0.8, 0.8]
    glTexParameterfv(GL_TEXTURE_2D, GL_TEXTURE_BORDER_COLOR, tex_border_color)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, 0x812F)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, 0x812F)
    
    
    glBegin(GL_POINTS)
    
    vertices = np.asanyarray(points.get_vertices())
    #tex_coords = np.asanyarray(points.get_texture_coordinates())
    
    for i in range(0, points.size()):
        if vertices[i][2]:
            glVertex3fv((vertices[i][0], vertices[i][1], vertices[i][2]))
            #glTexCoord2fv(tex_coords[i])
            
    glEnd()
    
    
    #glPopMatrix()
    #glMatrixMode(GL_PROJECTION)
    #glPopMatrix()
    #glPopAttrib()
    #glPushMatrix()
    

In [3]:
# Declare RealSense pipeline, encapsulating the actual device and sensors
pipeline = rs.pipeline()

config = rs.config()
config.enable_stream(rs.stream.depth, -1, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, -1, 640, 480, rs.format.rgb8, 30)

#Start streaming with default recommended configuration
cfg = pipeline.start(config)

align_to = rs.stream.color
align = rs.align(align_to)

dev = cfg.get_device()
depth_sensor = dev.first_depth_sensor()
depth_sensor.set_option(rs.option.visual_preset, 4)
depth_sensor.set_option(rs.option.enable_auto_exposure, 1)

# Declare pointcloud object, for calculating pointclouds and texture mappings
pc = rs.pointcloud()
# We want the points object to be persistent so we can display the last cloud when a frame drops
points = rs.points()

pygame.init()
display = (800,600)
pygame.display.set_mode(display, DOUBLEBUF|OPENGL)



appEvent = True

try:
    # Wait for the next set of frames from the camera
    frames = pipeline.wait_for_frames()
    aligned_frames = align.proccess(frames)

    # Fetch color and depth frames
    depth_frame = aligned_frames.get_depth_frame()
    color_frame = aligned_frames.get_color_frame()

    # Tell pointcloud object to map to this color frame
    pc.map_to(color_frame)

    # Generate the pointcloud and texture mappings
    points = pc.calculate(depth_frame)

    #print("Saving to 1.ply...")
    #points.export_to_ply("1.ply", color_frame)
    #print("Done")
    
    #vertices = np.asanyarray(points.get_vertices())
    #print(vertices[100])
    
    draw_pointcloud(points, 800, 600)

    while appEvent is True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                appEvent = False
            if event.type == pygame.KEYUP and event.key == pygame.K_ESCAPE:
                appEvent = False
        
        pygame.display.flip()
        pygame.time.wait(10)

finally:
    pipeline.stop()
    pygame.quit()